In [1]:
import pandas as pd
import tensorflow as tf

DATA_PATH = '/home/kshipra/work/major/ml experiments/data/manually_combined.xlsx'

eos = pd.read_excel(DATA_PATH, sheet_name='all_stacked_eos')
sentinel = pd.read_excel(DATA_PATH, sheet_name='all_stacked_sentinel')
combined = pd.read_excel(DATA_PATH, sheet_name='eos_sent_combined')

len(eos), len(sentinel), len(combined)

2025-09-26 21:18:07.282841: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(747, 796, 747)

In [2]:
combined = combined[combined['SM (Combined)'] < 150]

combined

,VH,VV,Angle,SM (Sent),HH,HV,SM (EOS),SM (Combined),Unnamed: 8,Latitude (S),Longitude (S),Latitude (E),Longitude (E),Lat Diff,Long Diff,SM Diff
0,-14.947200,-10.379400,40.366800,20.6,-7.38713,-15.09099,24.4,22.50,NaN,22.526048,72.765011,22.526048,72.765011,0.000000,0.000000,-3.8
1,-16.361200,-11.299400,41.008800,25.8,-6.25740,-12.05392,41.0,33.40,NaN,22.525481,72.765028,22.525481,72.765028,0.000000,0.000000,-15.2
2,-16.168800,-10.992000,42.359800,28.9,-8.44134,-16.46069,25.3,27.10,NaN,22.525423,72.765663,22.525423,72.765663,0.000000,0.000000,3.6
3,-15.630700,-8.829090,42.982700,4.1,-2.85467,-17.00499,53.7,28.90,NaN,22.525999,72.765663,22.525999,72.765663,0.000000,0.000000,-49.6
4,-15.130900,-10.333000,41.434700,35.3,-6.88868,-12.32378,53.4,44.35,NaN,22.525563,72.765399,22.525563,72.765399,0.000000,0.000000,-18.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,-15.603123,-10.557852,42.709682,30.2,-8.69274,-14.15688,26.8,28.50,NaN,22.505581,72.764064,22.523640,72.766727,-0.018059,-0.002663,3.4
743,-16.321512,-11.300431,42.176147,38.3,-6.48271,-15.56076,32.8,35.55,NaN,22.505961,72.764083,22.523657,72.766016,-0.017696,-0.001933,5.5
744,-15.585269,-11.489754,42.523727,20.0,-6.20660,-14.73141,34.8,27.40,NaN,22.506133,72.763434,22.524255,72.766024,-0.018122,-0.002590,-14.8
745,-16.052544,-10.954885,42.562408,39.7,-9.43414,-20.52987,27.8,33.75,NaN,22.505784,72.763859,22.524229,72.766601,-0.018445,-0.002742,11.9


In [3]:
X_cols = ['VH', 'VV', 'HH', 'HV', 'Angle']
y_col = ['SM (Combined)']

In [4]:
from itertools import combinations

all_feature_combinations = []
for i in range(1, len(X_cols) + 1):
    # Get all combinations of length 'i'
    combs = combinations(X_cols, i)
    all_feature_combinations.extend(list(combs))

print(f"Generated {len(all_feature_combinations)} feature combinations to test.")

Generated 31 feature combinations to test.


In [5]:
from  model_experiments import PredictionIntervalEstimation

all_results = []
seed = 1337 

for features in all_feature_combinations:
    features = list(features) # Convert tuple to list
    num_features = len(features)
    feature_string = ", ".join(features)

    print(f"\n--- Testing Features ({num_features}): {feature_string} ---")

    tf.keras.backend.clear_session()

    model = tf.keras.Sequential([
        tf.keras.Input(shape=(num_features, )), # Dynamic input shape
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.09),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dropout(0.09),
        tf.keras.layers.Dense(1)
    ])

    # Instantiate the experiment class with the current feature subset
    exp = PredictionIntervalEstimation(
        combined,
        features=features,
        target=y_col[0],
        satellite='EOS+Sentinel',
        random_state=seed
    )

    y_lower_test, y_upper_test, y_lower_val, y_upper_val = exp.train_model(
        model,
        learning_rate=0.0005, # Using the optimal LR from previous tuning
        epochs=1000,
        verbose=0 # Suppress epoch-by-epoch output
    )

    metrics = exp.evaluate_model(exp.y_val, y_lower_val, y_upper_val)

    result_entry = {
        'features': feature_string,
        'num_features': num_features,
        'PICP': metrics['PICP'],
        'MPIW': metrics['MPIW']
    }

    all_results.append(result_entry)

all_results


--- Testing Features (1): VH ---


Epochs:  13%|█▎        | 126/1000 [00:13<01:37,  9.01epoch/s, loss=0.4452, val_loss=0.4543]


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

--- Testing Features (1): VV ---


Epochs:  20%|██        | 204/1000 [00:21<01:23,  9.56epoch/s, loss=0.4362, val_loss=0.4480]

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

--- Testing Features (5): VH, VV, HH, HV, Angle ---


Epochs:  12%|█▏        | 121/1000 [00:14<01:44,  8.44epoch/s, loss=0.4186, val_loss=0.4372]

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


[{'features': 'VH',
  'num_features': 1,
  'PICP': np.float64(0.9583333333333334),
  'MPIW': np.float32(32.68273)},
 {'features': 'VV',
  'num_features': 1,
  'PICP': np.float64(0.9464285714285714),
  'MPIW': np.float32(36.081985)},
 {'features': 'HH',
  'num_features': 1,
  'PICP': np.float64(0.9464285714285714),
  'MPIW': np.float32(30.914167)},
 {'features': 'HV',
  'num_features': 1,
  'PICP': np.float64(0.9464285714285714),
  'MPIW': np.float32(32.56308)},
 {'features': 'Angle',
  'num_features': 1,
  'PICP': np.float64(0.9404761904761905),
  'MPIW': np.float32(32.52769)},
 {'features': 'VH, VV',
  'num_features': 2,
  'PICP': np.float64(0.9523809523809523),
  'MPIW': np.float32(34.54159)},
 {'features': 'VH, HH',
  'num_features': 2,
  'PICP': np.float64(0.9523809523809523),
  'MPIW': np.float32(30.34891)},
 {'features': 'VH, HV',
  'num_features': 2,
  'PICP': np.float64(0.9523809523809523),
  'MPIW': np.float32(30.527506)},
 {'features': 'VH, Angle',
  'num_features': 2,
  'PIC

In [6]:
results_df = pd.DataFrame(all_results)

results_df[results_df['PICP'] > 0.95].sort_values(by='MPIW', ascending=True)

,features,num_features,PICP,MPIW
19,"VH, HH, Angle",3,0.958333,29.158676
24,"HH, HV, Angle",3,0.952381,29.851120
28,"VH, HH, HV, Angle",4,0.958333,30.288589
6,"VH, HH",2,0.952381,30.348909
30,"VH, VV, HH, HV, Angle",5,0.958333,30.468517
7,"VH, HV",2,0.952381,30.527506
22,"VV, HH, Angle",3,0.952381,30.610973
25,"VH, VV, HH, HV",4,0.952381,30.739536
18,"VH, HH, HV",3,0.952381,30.829323
26,"VH, VV, HH, Angle",4,0.958333,31.244894


In [7]:
results_df

,features,num_features,PICP,MPIW
0,VH,1,0.958333,32.682732
1,VV,1,0.946429,36.081985
2,HH,1,0.946429,30.914167
3,HV,1,0.946429,32.563080
4,Angle,1,0.940476,32.527691
5,"VH, VV",2,0.952381,34.541592
6,"VH, HH",2,0.952381,30.348909
7,"VH, HV",2,0.952381,30.527506
8,"VH, Angle",2,0.958333,33.585594
9,"VV, HH",2,0.946429,32.088333
